<a href="https://colab.research.google.com/github/Alijgh003/StyleMind-GNN/blob/main/clipGarmentEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai-clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 830.6 kB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=fc0ffa1be35f425f061fcf88433c31d8bee534a61231ce892e43805a11cc63ca
  Stored in directory: /root/.cache/pip/wheels/0d/17/90/042948fd2e2a87f1dcf6db6d438cad015c49db0c53d1d9c7dc
Successfully built openai-clip


In [2]:
!pip install lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 7.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.transforms import InterpolationMode

import clip

import time
import lmdb
import io

from os import path

In [2]:
datasets_path = path.join("/","content","drive","MyDrive","StyleMind-GNN")
images_path = path.join(datasets_path, "dataset", "polyvore_outfits", "images")
edge_indices_path = path.join(datasets_path, "dataset", "polyvore_outfits", "nondisjoint")
lmdb_path = path.join(datasets_path, "dataset", "polyvore_outfits","my_dataset.lmdb")

In [3]:
import pandas as pd

In [4]:
items_df = pd.read_csv(path.join(datasets_path, "new_items.csv"))
items_df

,Unnamed: 0.1,Unnamed: 0,index,ID,url_name,category_id,semantic_category,fdescription
0,0,0,0,211990161,neck print chiffon plus size,3,tops,neck print chiffon plus size tops
1,1,1,1,183179503,christian pellizzari floral jacquard trousers,2,bottoms,christian pellizzari floral jacquard trousers ...
2,2,2,2,152771755,natures jewelry stainless steel not,11,jewellery,natures jewelry stainless steel not jewellery
3,3,3,3,190445143,balmain women high waist flared,2,bottoms,balmain women high waist flared Decorative gol...
4,4,4,4,211444470,acler orson high waist belted,2,bottoms,acler orson high waist belted This Acler Orson...
...,...,...,...,...,...,...,...,...
251003,251003,251003,251003,108411005,arnica embellished hand painted skirt,2,bottoms,arnica embellished hand painted skirt Shop Ste...
251004,251004,251004,251004,212452593,thom browne little boy knitted,4,outerwear,thom browne little boy knitted outerwear
251005,251005,251005,251005,208651882,colorful vortex print shawl scarf,8,scarves,colorful vortex print shawl scarf scarves
251006,251006,251006,251006,209210426,floral embroidered mesh see thru,3,tops,floral embroidered mesh see thru tops


In [5]:
def build_edge_indices_with_groups(edge_index_tensor_by_groups):
  if(edge_index_tensor_by_groups.shape[0]>0):
    result = torch.cat([edge_index_tensor_by_groups[i] for i in range(edge_index_tensor_by_groups.shape[0])],dim=-1)
    result = result[:,torch.where(result[0]>-1)[0]]
    return result
  else:
    return torch.randn((2, 1))

In [6]:
def build_edge_indices_and_indices_with_groups(edge_index_tensor_by_groups):
  if(edge_index_tensor_by_groups.shape[0]>0):
    edge_indices = torch.cat([edge_index_tensor_by_groups[i] for i in range(edge_index_tensor_by_groups.shape[0])],dim=-1)
    indices = torch.cat([torch.full((edge_index_tensor_by_groups[i].shape[-1],), i) for i in range(edge_index_tensor_by_groups.shape[0])], dim=-1)
    indices = indices[edge_indices[0]>=0].view(-1,1)
    edge_indices = edge_indices[edge_indices>=0].view(2,-1)
    return edge_indices, indices
  else:
    return torch.randn((2, 1)), torch.randn((2,1))

In [7]:
garments = items_df[['index', 'fdescription']].values.tolist()
garments = {i[0]:i[1] for i in garments}
garments

{0: 'neck print chiffon plus size     tops',
 1: "christian pellizzari floral jacquard trousers Gold and black silk blend floral jacquard trousers from Christian Pellizzari. Color: Metallic.  Gender: Female.  Pattern: Floral.  Material: Viscose/Polyester/Silk/Polyamide. Women's Fashion Clothing Pants Christian Pellizzari pants Christian Pellizzari floral jacquard trousers Floral pants Grey pants Print pants Patterned pants Floral-print pants Metallic pants bottoms",
 2: 'natures jewelry stainless steel not     jewellery',
 3: "balmain women high waist flared Decorative gold colored buttons with lion detail . High waisted . Flared hem. Sample size: 36 Women's Fashion Clothing Pants Balmain pants Balmain Women High Waist Flared Knit Pants Balmain Flared pants High-waisted pants Knit pants High rise pants Embellished pants bottoms",
 4: "acler orson high waist belted This Acler Orson High Waist Belted Short features a high waist, foldover detail, and a front self bow tie. Women's Fashion 

In [8]:
train_edge_index = torch.load(path.join(edge_indices_path, "train_edge_index.pth"))
negative_train_edge_index = torch.load(path.join(edge_indices_path,"negative_train_edge_index.pth"))

train_edge_index  = build_edge_indices_with_groups(train_edge_index)
negative_train_edge_index  = build_edge_indices_with_groups(negative_train_edge_index)

test_edge_index = torch.load(path.join(edge_indices_path, "test_edge_index.pth"))
negative_test_edge_index = torch.load(path.join(edge_indices_path, "negative_test_edge_index.pth"))

test_edge_index  = build_edge_indices_with_groups(test_edge_index)
negative_test_edge_index  = build_edge_indices_with_groups(negative_test_edge_index)



valid_edge_index = torch.load(path.join(edge_indices_path, "valid_edge_index.pth"))
negative_valid_edge_index = torch.load(path.join(edge_indices_path, "negative_valid_edge_index.pth"))

valid_edge_index  = build_edge_indices_with_groups(valid_edge_index)
negative_valid_edge_index  = build_edge_indices_with_groups(negative_valid_edge_index)


train_edge_index, negative_train_edge_index, valid_edge_index, negative_valid_edge_index, test_edge_index, negative_test_edge_index

(tensor([[204439, 154376, 154376,  ...,  74651,  74651,  74651],
         [249298, 249298, 204439,  ..., 145194, 153601, 220923]]),
 tensor([[118742, 118742, 100723,  ...,  64782,  64782,  64782],
         [178245, 100723, 178245,  ...,  37821,   4739,  57581]]),
 tensor([[ 67981,  67981,  67981,  ..., 121869, 121869, 121869],
         [221381, 112327,  39445,  ...,  36254,   9839, 118525]]),
 tensor([[102805,  61260, 234578,  ..., 124793, 124793, 122228],
         [176676, 176676, 176676,  ..., 122228, 142856, 142856]]),
 tensor([[121002,  69395,  69395,  ...,  72843, 185573, 185573],
         [238976, 121002, 238976,  ..., 185573,   7277,  99657]]),
 tensor([[ 74755,  63248, 223336,  ..., 211557, 211557, 128858],
         [243442, 243442, 243442,  ..., 128858,  67632,  67632]]))

In [9]:
train_edge_index.shape, test_edge_index.shape, valid_edge_index.shape

(torch.Size([2, 686835]), torch.Size([2, 129584]), torch.Size([2, 64922]))

In [10]:
build_edge_indices_and_indices_with_groups(train_edge_index)[:100]

(tensor([[204439, 154376, 154376,  ...,  74651,  74651,  74651],
         [249298, 249298, 204439,  ..., 145194, 153601, 220923]]),
 tensor([[0],
         [0],
         [0],
         ...,
         [1],
         [1],
         [1]]))

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import clip
from torch.utils.data import Dataset, DataLoader

In [12]:
class FusionHead(nn.Module):
    def __init__(self, embed_dim, num_of_layers=1):
        super(FusionHead, self).__init__()
        self.fcs = torch.nn.ModuleList([nn.Linear(2 * embed_dim, embed_dim)] + [nn.Linear(embed_dim, embed_dim) for _ in range(num_of_layers-1)])
        self.bns = torch.nn.ModuleList([nn.BatchNorm1d(embed_dim)] + [nn.BatchNorm1d(embed_dim) for _ in range(num_of_layers-1)])
        self.activation = nn.ReLU()

    def reset_parameters(self):
        for fc in self.fcs:
            fc.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for fc, bn in zip(self.fcs, self.bns):
            x = self.activation(bn(fc(x)))
        return x

In [13]:
class GarmentDataset(Dataset):
    def __init__(self, data, lmdb_path, transform=None, tokenizer= None):
        self.data = data # dict of (index: garment_description)
        self.env = lmdb.open(lmdb_path, readonly=True, lock=False, readahead=False)
        self.txn = self.env.begin(buffers=True)
        self.length = self._get_length()
        self.transform = transform
        self.tokenizer = tokenizer

    def _get_length(self):
        return len(self.data)

    def __getitem__(self, index):
        img_bytes = self.txn.get(str(index).encode('ascii'))
        if img_bytes is None:
            raise IndexError(f"Index {index} out of range")
        img = Image.open(io.BytesIO(img_bytes)).convert('RGB')
        if self.transform:
            img = self.transform(img)
        txt = self.data[index]
        if self.tokenizer:
            txt = tokenizer(txt, truncate=True)
        return img, txt

    def __len__(self):
        return self.length

In [14]:
class OutfitDataset(Dataset):
    def __init__(self, positive_pairs, negative_pairs, lmdb_dataset):
        #positive_pairs: (idx1, idx2)
        #negative_pairs: (idx1, idx2)

        self.lmdb_dataset = lmdb_dataset
        positive_pairs = [(i[0],i[1], 1) for i in positive_pairs]
        negative_pairs = [(i[0],i[1], 1) for i in negative_pairs]
        self.data = positive_pairs + negative_pairs

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        idx1, idx2, label = self.data[idx]
        return self.lmdb_dataset[idx1], self.lmdb_dataset[idx2], label

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [16]:
tokenizer = clip.tokenize

In [17]:
garment_dataset = GarmentDataset(data=garments, lmdb_path=lmdb_path,
                      transform=preprocess, tokenizer= tokenizer)

In [18]:
batch_size = 128

garment_data_loader = DataLoader(garment_dataset, batch_size=batch_size, num_workers=2, shuffle=True)

In [19]:
fusion_head = FusionHead(embed_dim=model.visual.output_dim).to(device)

In [20]:
from tqdm import tqdm

In [21]:
def save_model(model, dst_path):
  torch.save(model.state_dict(), dst_path)

In [36]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [22]:
class GarmentEmbedding(nn.Module):
  def __init__(self, clip_model, fusion_head):
    super(GarmentEmbedding, self).__init__()
    self.clip_model = clip_model
    self.fusion_head = fusion_head

  def forward(self, images, texts):
      image_emb = model.encode_image(images)
      text_emb = model.encode_text(texts)
      combined = torch.cat([image_emb, text_emb], dim=-1).float()
      emb = fusion_head(combined)
      emb = F.normalize(emb, p=2, dim=1)
      return emb

In [23]:
garment_embedding_model = GarmentEmbedding(model, fusion_head).to(device)

In [24]:
def get_garment_embedding(model, dataloader, epoch):
    embeddings = []
    for images, texts in tqdm(dataloader, desc=f"Epoch {epoch+1}", leave=False):
        images = images.to(device)
        texts = texts.squeeze(1).to(device)
        emb = model(images, texts)
        embeddings.append(emb)
    return torch.stack(embeddings)  # torch embedding tensor

def train(model, dataloader, positive_pairs, negative_pairs, epoch,loss_fn, optimizer):
    model.train()
    optimizer.zero_grad()
    embeddings = get_garment_embedding(model, dataloader, epoch)

    positive_labels = torch.ones(positive_pairs.shape[-1]).to(device)
    negative_labels = torch.zeros(negative_pairs.shape[-1]).to(device)

    labels = torch.cat((positive_labels,negative_labels), dim=-1)
    pairs = torch.cat((positive_pairs, negative_pairs), dim=-1)

    emb_a = embeddings[pairs[0]]
    emb_b = embeddings[pairs[1]]

    # Compute loss
    targets = 2 * labels - 1  # 1 for positive, -1 for negative
    loss = loss_fn(emb_a, emb_b, targets)

    # Optimize
    loss.backward()
    optimizer.step()

    return embeddings, loss.item()

def test(embeddings, positive_pairs, negative_pairs):
    accuracy = 0.0
    with torch.no_grad():
          positive_labels = torch.ones(positive_pairs.shape[-1]).to(device)
          negative_labels = torch.zeros(negative_pairs.shape[-1]).to(device)

          labels = torch.cat((positive_labels,negative_labels), dim=-1)
          pairs = torch.cat((positive_pairs, negative_pairs), dim=-1)

          emb_a = embeddings[pairs[0]]
          emb_b = embeddings[pairs[1]]

          similarities = F.cosine_similarity(dim=-1)
          corrects = (similarities[:positive_labels.shape[0]] >= 0).sum(dim=-1)
          corrects += (similarities[positive_labels.shape[0]:] < 0).sum(dim=-1)
          accuracy = (corrects / emb_a.shape[0]) * 100
    return accuracy

In [25]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(fusion_head.parameters()),
    lr=1e-5
)
loss_fn = nn.CosineEmbeddingLoss(margin=0.5)

best_valid_acc = 0.0

# ====================
# 6. Training Loop
# ====================
num_epochs = 10

for epoch in range(num_epochs):
    epoch_loss = 0.0
    # Wrap the data loader with tqdm and add a description
    embeddings, epoch_loss = train(garment_embedding_model, garment_data_loader, train_edge_index,
                   negative_train_edge_index, epoch ,loss_fn, optimizer)

    valid_acc = test(embeddings, valid_edge_index, negative_valid_edge_index)
    test_acc = test(embeddings, test_edge_index, negative_test_edge_index)

    if(valid_acc > best_valid_acc):
      save_model(garment_embedding_model, path.join(datasets_path, "garment_embd_clip.pth"))
      best_valid_acc = valid_acc

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss/len(garment_data_loader):.4f}\n valid_acc={valid_acc} \n test_acc={test_acc}")

print("Training complete!")

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 20.12 MiB is free. Process 41026 has 14.72 GiB memory in use. Of the allocated memory 14.10 GiB is allocated by PyTorch, and 500.84 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)